## Notebook for Constrastive Learning + linear alignment

### Import Libaries and download datafiles

In [4]:
!mkdir data
!gdown 1CVAQDuPOiwm8h9LJ8a_oOs6zOWS6EgkB
!gdown 1ykZ9fjTxUwdiEwqagoYZiMcD5aG-7rHe
!unzip -o test.zip -d data
!unzip -o train.zip -d data
!git clone https://github.com/Mamiglia/challenge.git

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: data/train/Images/4735742933.jpg  
  inflating: data/train/Images/3628829949.jpg  
  inflating: data/train/Images/608453511.jpg  
  inflating: data/train/Images/180027584.jpg  
  inflating: data/train/Images/1801283994.jpg  
  inflating: data/train/Images/6681942245.jpg  
  inflating: data/train/Images/3479423813.jpg  
  inflating: data/train/Images/2341755936.jpg  
  inflating: data/train/Images/2814406547.jpg  
  inflating: data/train/Images/4342753575.jpg  
  inflating: data/train/Images/489773343.jpg  
  inflating: data/train/Images/6556869643.jpg  
  inflating: data/train/Images/4644327815.jpg  
  inflating: data/train/Images/4957517466.jpg  
  inflating: data/train/Images/5384744603.jpg  
  inflating: data/train/Images/134894450.jpg  
  inflating: data/train/Images/3249014584.jpg  
  inflating: data/train/Images/3582920844.jpg  
  inflating: data/train/Images/5362687336.jpg  
  inflating: data/train/I

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from pathlib import Path
from tqdm import tqdm

from challenge.src.common import load_data, prepare_train_data, generate_submission

#### Data Augmentation

In [ ]:
# import argparse
# from PIL import Image
# import torch
# from sentence_transformers import SentenceTransformer
# from transformers import AutoImageProcessor, AutoModel
# from tqdm import tqdm
# import numpy as np
# import pandas as pd
# from pathlib import Path
# from torchvision import transforms
# import random
# # roberta-large-nli-stsb-mean-tokens
# def load_text_model(model_name="sentence-transformers/roberta-large-nli-stsb-mean-tokens"):
#     """Load Sentence-BERT text encoder."""
#     print(f"Loading text model: {model_name}")
#     return SentenceTransformer(model_name)


# def load_image_model(model_name="facebook/dinov2-giant"):
#     """Load DINOv2 image encoder."""
#     print(f"Loading image model: {model_name}")
#     image_processor = AutoImageProcessor.from_pretrained(model_name, use_fast=True)
#     model = AutoModel.from_pretrained(model_name)
#     return image_processor, model

# ## Can be modified for data augmentation if needed or create my one
# def get_augmentations(num_augmentations=3):
#     """Create a list of augmentation pipelines (light + heavy mix)."""
#     light = transforms.Compose([
#         transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
#         transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
#         transforms.RandomHorizontalFlip(),
#     ])

#     heavy = transforms.Compose([
#         transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
#         transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
#         transforms.RandomGrayscale(p=0.2),
#         transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
#         transforms.RandomHorizontalFlip(),
#     ])

#     augs = []
#     for _ in range(num_augmentations):
#         augs.append(random.choice([light, heavy]))
#     return augs

# @torch.inference_mode()
# def process_images_batch(image_processor, model, image_paths, device, batch_size=128, dataset_path=None, augmentations=3):
#     """Generate image embeddings in batches."""
#     print(f"Processing {len(image_paths)} images in batches...")
#     model.to(device)
#     model.eval()

#     all_embeddings = []
#     all_names = []
#     img_files = []
#     augs = get_augmentations(augmentations)
#     all_images = []

#     for i in tqdm(range(0, len(image_paths), batch_size), desc="Encoding images"):
#         batch_paths = image_paths[i:i+batch_size]
#         valid_images = []

#         # Keep track of which original indices correspond to valid images in the batch
#         valid_paths = []

#         for j, path in enumerate(batch_paths):
#             original_index = i + j
#             try:
#                 img = Image.open(dataset_path / 'Images' / path).convert("RGB")
#                 valid_images.append(img)
#                 valid_paths.append(path)
#                 for k, aug in enumerate(augs):
#                   aug_img = aug(img)
#                   valid_images.append(aug_img)
#                   valid_paths.append(f"{path}_aug{k+1}")
#             except Exception as e:
#                 print(f"Warning: Skipping image {path} due to error: {e}")

#         if not valid_images:
#             continue

#         inputs = image_processor(images=valid_images, return_tensors="pt").to(device)
#         outputs = model(**inputs)

#         # Average over patch tokens
#         image_features = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

#         # Store embeddings based on their success
#         all_embeddings.extend(image_features)
#         img_files.extend(valid_paths)

#     all_embeddings = np.vstack(all_embeddings)

#     return all_names, all_embeddings


# def process_captions(text_model, captions, device):
#     """Generate text embeddings using Sentence-BERT."""
#     print("Processing captions...")
#     return text_model.encode(
#         captions,
#         convert_to_numpy=True,
#         show_progress_bar=True,
#         device=device
#     )

# def load_dataset(dataset_path):
#     """
#     Load dataset from a directory containing captions.txt and an Images folder.
#     """
#     captions_file = dataset_path / "captions.txt"
#     images_dir = dataset_path / "Images"
#     if not captions_file.exists() or not images_dir.is_dir():
#         raise FileNotFoundError(f"Could not find 'captions.txt' or 'Images' directory in {dataset_path}")

#     df = pd.read_csv(captions_file)

#     if 'id' not in df.columns:
#         df['id'] = np.arange(len(df))

#     return df

# num_augmentations = 4
# dataset_path = Path('drive/MyDrive/data/train/')
# device = "cuda" if torch.cuda.is_available() else "cpu"
# output_file = 'drive/MyDrive/data/augmented_testdata.npz'
# print(f"Using device: {device}")

# print(f"Loading dataset from: {dataset_path}")
# df_captions = load_dataset(dataset_path)

# text_model = load_text_model()
# image_processor, image_model = load_image_model()

# all_captions = df_captions['caption'].tolist()
# caption2img = df_captions['image'].tolist()
# all_images = df_captions['image'].unique().tolist()

# num_images = len(all_images)
# num_captions = len(all_captions)
# print(f"Found {num_images} images and {num_captions} total captions.")

# all_images, img_embd = process_images_batch(image_processor,
#                                             image_model,
#                                             all_images,
#                                             device,
#                                             dataset_path=dataset_path, augmentations=num_augmentations)

# images_dict = {img_name: i for i, img_name in enumerate(all_images)}


Using device: cuda
Loading dataset from: drive/MyDrive/data/train
Loading text model: sentence-transformers/roberta-large-nli-stsb-mean-tokens
Loading image model: facebook/dinov2-giant
Found 25000 images and 125000 total captions.
Processing 128 images in batches...


Encoding images:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Duplicate captions per image *and* per augmentation
# repeated_captions = []
# repeated_imgnames = []
# num_captions_per_image = 5
# num_versions_per_image = 1 + num_augmentations  # original + 3 augmentations

# for i in range(0, len(all_images), num_versions_per_image):
#     image_group = all_images[i : i + num_versions_per_image]
#     for j in range(num_captions_per_image):
#         caption = all_captions[(i // num_versions_per_image) * num_captions_per_image + j]
#         for img_variant in image_group:
#             repeated_captions.append(caption)
#             repeated_imgnames.append(img_variant)


# print(len(repeated_captions), len((repeated_imgnames)))
# caption_embeddings = process_captions(text_model, repeated_captions, device)

# num_images = len(all_images)
# num_captions = len(repeated_captions)

# label = np.zeros((num_captions, num_images), dtype=bool)
# image_idx_map = {name: i for i, name in enumerate(all_images)}
# for idx, img_name in enumerate(repeated_imgnames):
#     if img_name in image_idx_map:
#         label[idx, image_idx_map[img_name]] = 1

# data = {
#     'metadata/num_captions': np.array([num_captions]),
#     'metadata/num_images': np.array([num_images]),
#     'metadata/embedding_dim_text': np.array([caption_embeddings.shape[1]]),
#     'metadata/embedding_dim_image': np.array([img_embd.shape[1]]),
#     'captions/text': np.array(repeated_captions),
#     'captions/embeddings': caption_embeddings,
#     'captions/label': label,
#     'images/names': np.array(all_images),
#     'images/embeddings': img_embd,
# }

# print(f"Saving augmented dataset to {output_file}")
# np.savez_compressed(output_file, **data)
# print("✓ Done.")

### Create Neural Network Architectures

In [6]:
class TransformerTranslator(nn.Module):
    """
    Transformer-style translator from text embedding -> image embedding
    """
    def __init__(self, text_dim=1024, img_dim=1536, n_heads=8, n_layers=2, dim_feedforward=2048, dropout=0.2):
        super().__init__()
        self.input_ln = nn.LayerNorm(text_dim)
        self.proj_in = nn.Linear(text_dim, img_dim)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=img_dim,
            nhead=n_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.output_ln = nn.LayerNorm(img_dim)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.input_ln(x)
        x = self.proj_in(x)
        out = self.encoder(x)
        out = out.squeeze(1)
        return self.output_ln(out)

In [7]:
class ResidualMLPTranslator(nn.Module):
    def __init__(self, text_dim=1024, img_dim=1536, hidden_dim=2048, num_layers=3, dropout=0.2):
        super().__init__()
        assert num_layers >= 2
        self.input_ln = nn.LayerNorm(text_dim)
        self.dropout = nn.Dropout(dropout)

        self.first_layer = nn.Sequential(
            nn.Linear(text_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout)
        )

        self.blocks = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.GELU(),
                nn.LayerNorm(hidden_dim),
                nn.Dropout(dropout)
            )
            for _ in range(num_layers - 2)
        ])

        self.final_proj = nn.Linear(hidden_dim, img_dim)
        self.output_ln = nn.LayerNorm(img_dim)

        if text_dim != img_dim:
            self.res_proj = nn.Linear(text_dim, img_dim)
        else:
            self.res_proj = nn.Identity()

    def forward(self, x):
        x_in = self.input_ln(x)
        out = self.first_layer(x_in)
        for block in self.blocks:
            out = out + block(out)
        out = self.final_proj(out)
        out = out + self.res_proj(x_in)
        return self.output_ln(out)


In [9]:
class LatentSpaceTranslator(nn.Module):
    def __init__(self,
                 text_dim=1024,
                 img_dim=1536,
                 hidden_dim=2048,
                 num_layers=3,
                 dropout=0.2,
                 use_residual=True):
        super().__init__()
        self.use_residual = use_residual
        self.input_ln = nn.LayerNorm(text_dim)
        layers = []
        in_dim = text_dim
        for i in range(num_layers - 1):
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.LayerNorm(hidden_dim))
            layers.append(nn.Dropout(dropout))
            in_dim = hidden_dim
        # final projection to image space
        layers.append(nn.Linear(in_dim, img_dim))
        self.net = nn.Sequential(*layers)

        if self.use_residual:
            if text_dim != img_dim:
                self.res_proj = nn.Linear(text_dim, img_dim)
            else:
                self.res_proj = nn.Identity()

        # final layer norm in image space
        self.output_ln = nn.LayerNorm(img_dim)

    def forward(self, text_emb):
        if text_emb.dim() == 3:
            x = text_emb.squeeze(1)
        else:
            x = text_emb
        x = self.input_ln(x)
        out = self.net(x)
        if self.use_residual:
            res = self.res_proj(x)
            out = out + res
        return self.output_ln(out)


### Training Loop and NCE Loss aswell as Procrustes Init

In [10]:

class QueueInfoNCELoss(nn.Module):
    def __init__(self, dim, temperature=0.07, queue_size=4096):
        super().__init__()
        self.temperature = temperature
        self.queue_size = queue_size
        self.register_buffer("queue", torch.randn(queue_size, dim))
        self.queue = F.normalize(self.queue, dim=1)
        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _enqueue(self, keys):
        batch_size = keys.shape[0]
        ptr = int(self.queue_ptr.item())
        end_ptr = (ptr + batch_size) % self.queue_size

        if end_ptr > ptr:
            self.queue[ptr:end_ptr] = keys
        else:
            # wrap
            first_len = self.queue_size - ptr
            self.queue[ptr:] = keys[:first_len]
            self.queue[:end_ptr] = keys[first_len:]
        self.queue_ptr[0] = end_ptr

    def forward(self, z_i, z_j):
        # normalize
        z_i = F.normalize(z_i, dim=1)
        z_j = F.normalize(z_j, dim=1)

        # positive logits: (B, 1)
        l_pos = torch.sum(z_i * z_j, dim=-1, keepdim=True)

        # negative logits from queue: (B, queue_size)
        l_neg = torch.matmul(z_i, self.queue.T)

        # logits: (B, 1 + queue_size)
        logits = torch.cat([l_pos, l_neg], dim=1)
        logits /= self.temperature

        labels = torch.zeros(logits.size(0), dtype=torch.long, device=z_i.device)

        loss = F.cross_entropy(logits, labels)
        return loss


In [11]:
# ====== Procrustes initialization ======
def procrustes_init(text_embs, img_embs):
    """
    text_embs: (N, d_text)
    img_embs:  (N, d_img)
    returns: weight matrix (d_img, d_text)
    """
    # Center both
    X = text_embs - text_embs.mean(0, keepdim=True)
    Y = img_embs - img_embs.mean(0, keepdim=True)

    # Compute SVD of cross-covariance
    U, _, Vt = torch.linalg.svd(X.T @ Y, full_matrices=False)
    W = U @ Vt
    return W.T   # shape (d_img, d_text) for nn.Linear weight


def apply_procrustes_init_to_final(model, text_sample, img_sample):
    """
    Apply Procrustes initialization to a model.
    - For MLP / ResidualMLP: apply to final Linear layer (hidden -> img_dim)
    - For TransformerTranslator: apply to first projection (text_dim -> img_dim)
    """
    with torch.no_grad():
        # Compute Procrustes matrix
        W = procrustes_init(text_embs=text_sample, img_embs=img_sample)

        # Apply to the appropriate layer
        applied = False
        for name, m in model.named_modules():
            if isinstance(m, nn.Linear):
                # Transformer: apply to first projection (proj_in)
                if isinstance(model, TransformerTranslator) and name.endswith("proj_in"):
                    print(m.weight.shape, W.shape)
                    if m.weight.shape == W.shape:
                        m.weight.copy_(W)
                        applied = True
                        break
                # MLP / ResidualMLP: apply to final_proj
                elif isinstance(model, LatentSpaceTranslator) and name.endswith("res_proj"):
                    print(m.weight.shape, W.shape)
                    if m.weight.shape == W.shape:
                        m.weight.copy_(W)
                        applied = True
                        break

                elif isinstance(model, ResidualMLPTranslator) and name.endswith("res_proj"):
                    print(m.weight.shape, W.shape)
                    if m.weight.shape == W.shape:
                        m.weight.copy_(W)
                        applied = True
                        break

        if not applied:
            print("⚠️ Warning: Could not find matching layer for Procrustes init")
    return model


In [12]:
def training(model, train_loader, val_loader, device, epochs, lr, MODEL_PATH,
             use_procrustes_init=True, procrustes_subset=10000, temperature=0.07,
             queue_size=4098):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-3)
    best_val_loss = float('inf')

    if use_procrustes_init:
        print("Computing Procrustes initialization...")
        text_list, img_list = [], []
        for i, (X, y) in enumerate(train_loader):
            text_list.append(X.cpu())
            img_list.append(y.cpu())
            if sum(t.shape[0] for t in text_list) >= procrustes_subset:
                break
        text_sample = torch.cat(text_list, dim=0)[:procrustes_subset]
        img_sample = torch.cat(img_list, dim=0)[:procrustes_subset]
        model = apply_procrustes_init_to_final(model, text_sample, img_sample)

    criterion = QueueInfoNCELoss(dim=1536, temperature=temperature, queue_size=queue_size).to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            pred = model(X_batch)
            loss = criterion(pred, y_batch)
            loss += 0.1 * F.mse_loss(pred, y_batch)
            loss += 1 - F.cosine_similarity(
                F.normalize(pred, dim=-1),
                F.normalize(y_batch, dim=-1)
            ).mean()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()

            with torch.no_grad():
              keys = F.normalize(y_batch, dim=1).detach()
              criterion._enqueue(keys)

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                pred = model(X_batch)
                loss = criterion(pred, y_batch)
                loss += 1 - F.cosine_similarity(
                    F.normalize(pred, dim=-1),
                    F.normalize(y_batch, dim=-1)
                ).mean()
                loss += 0.1 * F.mse_loss(pred, y_batch)
                val_loss += loss.item()

                keys = F.normalize(y_batch, dim=1).detach()
                criterion._enqueue(keys)


        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}: Train Loss = {train_loss:.6f}, Val Loss = {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            Path(MODEL_PATH).parent.mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"  ✓ Saved best model (val_loss={val_loss:.6f})")

    return model

### Load the Data

In [13]:
# 4. Data Augmentation
# 5. Zero Shot Stitching
# 6. Triplet Loss / Improve InfoNCE Loss / bidirectional / SimCLR / MoCo
# 7. Autoencoder
# Configuration
EPOCHS = 100
BATCH_SIZE = 512
LR = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load data
train_data = load_data("data/train/train.npz")
#train_data = load_data('processed_augmented_data.npz')
X, y, label = prepare_train_data(train_data)
DATASET_SIZE = len(X)
# Split train/val
# This is done only to measure generalization capabilities, you don't have to
# use a validation set (though we encourage this)
n_train = int(0.9 * len(X))
TRAIN_SPLIT = torch.zeros(len(X), dtype=torch.bool)
TRAIN_SPLIT[:n_train] = 1
X_train, X_val = X[TRAIN_SPLIT], X[~TRAIN_SPLIT]
y_train, y_val = y[TRAIN_SPLIT], y[~TRAIN_SPLIT]

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
y_train.shape, X_train.shape, train_loader.batch_size, val_loader.batch_size

(125000,)
Train data: 125000 captions, 125000 images


(torch.Size([112500, 1536]), torch.Size([112500, 1024]), 512, 512)

### Hyperparameter Optimization

In [14]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 29.3 MB/s eta 0:00:00


In [18]:
import optuna

def objective_extended(arch, trial, train_dataloader, val_dataloader, device, MODEL_PATH_BASE):

    # --- Common hyperparameters ---
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-3)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)

    # --- New hyperparameters ---
    temperature = trial.suggest_float("temperature", 0.01, 0.2)
    queue_size = trial.suggest_categorical("queue_size", [2048, 4098, 8196])
    procrustes_subset = 10000

    if arch in ["MLP", "ResidualMLP"]:
        hidden_dim = trial.suggest_categorical("hidden_dim", [512,1024, 2048, 4096])
        num_layers = trial.suggest_int("num_layers", 2, 6)
        if arch == "MLP":
            model = LatentSpaceTranslator(
                text_dim=1024, img_dim=1536, hidden_dim=hidden_dim,
                num_layers=num_layers, dropout=dropout
            ).to(device)
        else:
            model = ResidualMLPTranslator(
                text_dim=1024, img_dim=1536, hidden_dim=hidden_dim,
                num_layers=num_layers, dropout=dropout
            ).to(device)
    elif arch == "Transformer":
        n_layers = trial.suggest_int("n_layers", 2, 6)
        n_heads = trial.suggest_categorical("n_heads", [4, 8, 12])
        dim_feedforward = trial.suggest_categorical("dim_feedforward", [1024, 2048, 4096])
        model = TransformerTranslator(
            text_dim=1024, img_dim=1536,
            n_heads=n_heads, n_layers=n_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        ).to(device)

    if procrustes_subset > 0:
        # Get subset from train_loader
        text_list, img_list = [], []
        for i, (X, y) in enumerate(train_loader):
            text_list.append(X.cpu())
            img_list.append(y.cpu())
            if sum(t.shape[0] for t in text_list) >= procrustes_subset:
                break
        text_sample = torch.cat(text_list, dim=0)[:procrustes_subset]
        img_sample = torch.cat(img_list, dim=0)[:procrustes_subset]
        model = apply_procrustes_init_to_final(model, text_sample, img_sample)

    criterion = QueueInfoNCELoss(dim=1536, temperature=temperature, queue_size=queue_size).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    model.train()
    torch.autograd.set_detect_anomaly(True)

    for epoch in range(10):
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            pred = model(X_batch)

            # Weighted combination of losses
            loss = criterion(pred, y_batch)
            loss += 0.1 * F.mse_loss(pred, y_batch)
            loss += 1 - F.cosine_similarity(
                F.normalize(pred, dim=-1),
                F.normalize(y_batch, dim=-1)
            ).mean()

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            with torch.no_grad():
              keys = F.normalize(y_batch, dim=1).detach()
              criterion._enqueue(keys)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            pred = model(X_batch)
            loss =  criterion(pred, y_batch)
            loss += 0.1 * F.mse_loss(pred, y_batch)
            loss += 1 - F.cosine_similarity(
                F.normalize(pred, dim=-1),
                F.normalize(y_batch, dim=-1)
            ).mean()

            val_loss += loss.item()

            keys = F.normalize(y_batch, dim=1).detach()
            criterion._enqueue(keys)

    val_loss /= len(val_loader)

    return val_loss


def run_optuna_extended(arch, train_dataloader, val_dataloader, device, MODEL_PATH_BASE, n_trials=5):
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective_extended(arch, trial, train_dataloader, val_dataloader, device, MODEL_PATH_BASE),
                   n_trials=n_trials)

    print("Best trial:")
    trial = study.best_trial
    print(f"Val loss: {trial.value}")
    print("Best hyperparameters:")
    for key, value in trial.params.items():
        print(f"  {key}: {value}")

    return trial.params

In [20]:
archs = ['MLP', 'ResidualMLP', 'Transformer']
choosen_arch = archs[0]
best_params = run_optuna_extended(
    arch = choosen_arch,
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    device=DEVICE,
    MODEL_PATH_BASE="models/translator_optuna",
    n_trials=50
)

[I 2025-11-18 19:55:39,218] A new study created in memory with name: no-name-61d342f8-645c-482b-959c-18ce9b89b273
/tmp/ipython-input-3557271749.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 5e-4, 1e-3)
/tmp/ipython-input-3557271749.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)


torch.Size([1536, 1024]) torch.Size([1536, 1024])


[I 2025-11-18 19:57:14,667] Trial 0 finished with value: 3.2461322212219237 and parameters: {'dropout': 0.3929783382274049, 'lr': 0.000603046039065647, 'weight_decay': 0.0009185715586796968, 'temperature': 0.028539284956939748, 'queue_size': 2048, 'hidden_dim': 2048, 'num_layers': 4}. Best is trial 0 with value: 3.2461322212219237.


torch.Size([1536, 1024]) torch.Size([1536, 1024])


[I 2025-11-18 20:00:47,793] Trial 1 finished with value: 7.010941123962402 and parameters: {'dropout': 0.4290322405981011, 'lr': 0.0008891575544553996, 'weight_decay': 2.121083274858235e-05, 'temperature': 0.10914251530100925, 'queue_size': 8196, 'hidden_dim': 4096, 'num_layers': 5}. Best is trial 0 with value: 3.2461322212219237.


...

torch.Size([1536, 1024]) torch.Size([1536, 1024])


[I 2025-11-18 20:03:27,407] Trial 3 finished with value: 5.025633907318115 and parameters: {'dropout': 0.18467051726891223, 'lr': 0.0006343895571365492, 'weight_decay': 2.5890274115374223e-05, 'temperature': 0.05860622216591469, 'queue_size': 4098, 'hidden_dim': 512, 'num_layers': 2}. Best is trial 0 with value: 3.2461322212219237.


torch.Size([1536, 1024]) torch.Size([1536, 1024])


[I 2025-11-18 20:04:36,276] Trial 4 finished with value: 6.578538684844971 and parameters: {'dropout': 0.1767775044093623, 'lr': 0.0005818639039213856, 'weight_decay': 0.00029776235588465353, 'temperature': 0.19824973150617856, 'queue_size': 2048, 'hidden_dim': 2048, 'num_layers': 2}. Best is trial 0 with value: 3.2461322212219237.


Best trial:
Val loss: 3.2461322212219237
Best hyperparameters:
  dropout: 0.3929783382274049
  lr: 0.000603046039065647
  weight_decay: 0.0009185715586796968
  temperature: 0.028539284956939748
  queue_size: 2048
  hidden_dim: 2048
  num_layers: 4


### Training and Submission File Creation

In [21]:
if choosen_arch == 'Transformer':
    model = TransformerTranslator(
        text_dim=1024,
        img_dim=1536,
        n_heads = best_params['n_heads'],
        n_layers=best_params['n_layers'],
        dim_feedforward=best_params['dim_feedforward'],
        dropout=best_params['dropout']
    ).to(DEVICE)
    MODEL_PATH = "drive/MyDrive/data//models/transformer.pth"

elif choosen_arch == 'MLP':
    model = LatentSpaceTranslator(
    text_dim=1024,
    img_dim=1536,
    hidden_dim=best_params["hidden_dim"],
    num_layers=best_params["num_layers"],
    dropout=best_params["dropout"]).to(DEVICE)
    MODEL_PATH = "drive/MyDrive/data/models/latent_space.pth"

else:
    model = ResidualMLPTranslator(
    text_dim=1024,
    img_dim=1536,
    hidden_dim=best_params["hidden_dim"],
    num_layers=best_params["num_layers"],
    dropout=best_params["dropout"]).to(DEVICE)
    MODEL_PATH = "drive/MyDrive/data/models/residual.pth"

In [22]:
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Train
print("\n3. Training...")
model = training(model,
                 train_loader,
                 val_loader,
                 DEVICE,
                 EPOCHS,
                 best_params["lr"],
                 MODEL_PATH,
                 True,
                 10000,
                 best_params["temperature"],
                 best_params["queue_size"])

   Parameters: 15,230,976

3. Training...
Computing Procrustes initialization...
torch.Size([1536, 1024]) torch.Size([1536, 1024])


Epoch 1/100: 100%|██████████| 220/220 [00:08<00:00, 24.72it/s]


Epoch 1: Train Loss = 3.812735, Val Loss = 3.611962
  ✓ Saved best model (val_loss=3.611962)


Epoch 2/100: 100%|██████████| 220/220 [00:09<00:00, 24.11it/s]


Epoch 2: Train Loss = 3.308755, Val Loss = 3.468462
  ✓ Saved best model (val_loss=3.468462)


...

Epoch 14: Train Loss = 2.104370, Val Loss = 3.238830


Epoch 15/100: 100%|██████████| 220/220 [00:08<00:00, 24.88it/s]


Epoch 15: Train Loss = 2.064285, Val Loss = 3.239895


Epoch 16/100:  67%|██████▋   | 148/220 [00:06<00:03, 22.87it/s]


KeyboardInterrupt: 

In [23]:
model.load_state_dict(torch.load(MODEL_PATH))
test_data = load_data("data/test/test.clean.npz")

test_embds = test_data['captions/embeddings']
test_embds = torch.from_numpy(test_embds).float()

with torch.no_grad():
    pred_embds = model(test_embds.to(DEVICE)).cpu()

submission = generate_submission(test_data['captions/ids'], pred_embds, f'{choosen_arch}_submission.csv')
print(f"Model saved to: {MODEL_PATH}")

Generating submission file...
✓ Saved submission to MLP_submission.csv
Model saved to: drive/MyDrive/data/models/latent_space.pth
